# physical activity

In [1]:
import yaml
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
import re
import stemgraphic
import functools

from pathlib import Path
from itertools import compress
from sklearn.cluster import KMeans
from scipy import signal

%matplotlib inline

Paths to phenotype data and project directories

In [2]:
wrk_dir = Path('.')

with open(wrk_dir / 'config/path.yaml') as f:
    path = yaml.full_load(f)

app_dir = Path(path['application'])
prj_dir = Path(path['project'])

phe_dir = app_dir / 'phenotypes'

Read field look-up table and phenotype data. (This data was prepared using ukbkings, R/fields.R)

In [3]:
field_files = glob.glob(str(phe_dir / '*field_finder.txt'))

f = pd.concat((pd.read_csv(f, sep='\t', header=0) for f in field_files), ignore_index=True)

col_type = {
        'Sequence': 'Int64',
        'Integer': 'Int64',
        'Categorical (single)': 'str',
        'Categorical (multiple)': 'str',
        'Continuous': 'float',
        'Text': 'str',
        'Date': 'str',
        'Time': 'str',
        'Compound': 'str',
        'Binary object': 'str',
        'Records': 'str',
        'Curve': 'str'
}

f['py_type'] = [col_type[t] for t in f['ukb_type']]

df = pd.read_csv(prj_dir / 'data/pheno/crf.pa.csv',
                 dtype=dict(zip(f['field'], f['py_type'])))

# Update columns to descriptive names
fields = df.columns.tolist()
f = f.loc[f.field.isin(fields), ].drop_duplicates()
df.columns = np.ravel([f.loc[f.field == col, 'name'].tolist() for col in df.columns]).tolist()

# Clean up names
char_to_replace = {'(': '', ')': '', '/': '_'}
df.columns = [name.translate(str.maketrans(char_to_replace)) for name in df.columns.to_list()]

## PA phenotype QC

Read fam, sample, and sqc files for genetic metadata sample QC

In [4]:
fam_col_names = ['fid', 'iid', 'pid', 'mid', 'sex', 'phe']
fam = pd.read_csv(app_dir / 'genotyped/wukb13427_cal_chr22_v2_s488264.fam',
                  sep=' ', header=None, names=fam_col_names)

sample_col_names = ['fid', 'iid', 'miss', 'sex']
sample_file = pd.read_csv(app_dir / 'imputed/wukb13427_imp_chr22_v3_s487296.sample',
                          sep=' ', header=None, names=sample_col_names,
                          skiprows=[0, 1])

sqc_col_names = list(
    ['x1', 'x2', 'genotyping.array', 'Batch', 'Plate.Name', 'Well', 'Cluster.CR', 'dQC', 'Internal.Pico..ng.uL.',
     'Submitted.Gender', 'Inferred.Gender', 'X.intensity', 'Y.intensity', 'Submitted.Plate.Name', 'Submitted.Well',
     'sample.qc.missing.rate', 'heterozygosity', 'heterozygosity.pc.corrected', 'het.missing.outliers',
     'putative.sex.chromosome.aneuploidy', 'in.kinship.table', 'excluded.from.kinship.inference', 'excess.relatives',
     'in.white.British.ancestry.subset', 'used.in.pca.calculation'] +
    [f'{a}{b}' for a, b in zip(['pc'] * 40, list(range(1, 40 + 1)))] + # 26:65
    ['in.Phasing.Input.chr1_22', 'in.Phasing.Input.chrX', 'in.Phasing.Input.chrXY'])

sqc_col_names = [col_name.lower().replace('.', '_') for col_name in sqc_col_names] 
sqc = pd.read_csv(app_dir / 'imputed/ukb_sqc_v2.txt', sep=' ', header=None, names=sqc_col_names)

# Identify EUR sample by 4-means clustering
model = KMeans(4).fit(sqc.loc[:, ['pc1', 'pc2']])
sqc['pop'] = model.labels_
sqc = sqc[['genotyping_array', 'submitted_gender', 'inferred_gender',
           'het_missing_outliers', 'putative_sex_chromosome_aneuploidy',
           'excess_relatives', 'pop']]

# BOLT-LMM requires sample to be in both fam and sample files
meta = pd.merge(sample_file[['fid', 'iid']], fam.join(sqc), on=('fid', 'iid'), how='inner')

df = pd.merge(df, meta, left_on='eid', right_on='fid', how='right')

Filter and select variables

In [5]:
pa = df[(
    (df['data_quality_good_wear_time_f90015_0_0'] == '1') &
    (df['data_quality_good_calibration_f90016_0_0'] == '1') &
    df['data_problem_indicator_f90002_0_0'].isna() &
    df['no_wear_time_bias_adjusted_average_acceleration_f90087_0_0'].notna() &
    (df['no_wear_time_bias_adjusted_average_acceleration_f90087_0_0'] < 100) &
    (df['submitted_gender'] == df['inferred_gender']) &
    (df['excess_relatives'] == 0) &
    (df['putative_sex_chromosome_aneuploidy'] == 0) &
    (df['het_missing_outliers'] == 0) &
    (df['pop'] == df['pop'].value_counts().idxmax())
    )]

In [6]:
pa = pa[['eid',
         'no_wear_time_bias_adjusted_average_acceleration_f90087_0_0',
#          'age_when_attended_assessment_centre_f21003',
         'age_at_recruitment_f21022_0_0',
         'inferred_gender',
         'body_mass_index_bmi_f21001_0_0',
         'genotyping_array',
         'uk_biobank_assessment_centre_f54_0_0',
         'smoking_status_f20116_0_0',
         'alcohol_drinker_status_f20117_0_0',]]

pa = pa.rename({'age_at_recruitment_f21022_0_0': 'age',
                'inferred_gender': 'sex',
                'body_mass_index_bmi_f21001_0_0': 'bmi',
                'genotyping_array': 'array',
                'uk_biobank_assessment_centre_f54_0_0': 'centre',
                'smoking_status_f20116_0_0': 'smoking',
                'alcohol_drinker_status_f20117_0_0': 'alcohol',
                'no_wear_time_bias_adjusted_average_acceleration_f90087_0_0': 'pa'},
               axis='columns')

In [7]:
# BOLT-LMM pheno file requires header, with first 2 columns FID IID
pa['FID'] = pa['eid']
pa = pa.rename(columns={'eid': 'IID'})

pa = pa.replace({'sex': {'M': '1', 'F': '2'},
                 'array': {'UKBL': '0', 'UKBB': '1'},
                 'smoking': {'-3': 'NA'},
                 'alcohol': {'-3': 'NA'}})

pa['mpa'] =  np.where(pa['sex']=='1', pa['pa'], np.nan)
pa['fpa'] =  np.where(pa['sex']=='2', pa['pa'], np.nan)

In [8]:
# stemgraphic.stem_graphic(pa['no_wear_time_bias_adjusted_average_acceleration_f90087_0_0'])

Write phenotype/ covariate and keep files

In [9]:
(pa[pa['pa'].notnull()][['FID', 'IID', 'pa', 'age', 'sex', 'array', 'centre', 'bmi', 'smoking', 'alcohol']]
 .to_csv('data/pheno/pa.pheno', sep=' ', header=True, index=False, na_rep='NA'))

(pa[pa['mpa'].notnull()][['FID', 'IID', 'pa', 'age', 'sex', 'array', 'centre', 'bmi', 'smoking', 'alcohol']]
 .to_csv('data/pheno/pa.m.pheno', sep=' ', header=True, index=False, na_rep='NA'))

(pa[pa['fpa'].notnull()][['FID', 'IID', 'pa', 'age', 'sex', 'array', 'centre', 'bmi', 'smoking', 'alcohol']]
 .to_csv('data/pheno/pa.f.pheno', sep=' ', header=True, index=False, na_rep='NA'))

pa[['FID', 'IID']].to_csv('data/pheno/pa.keep', sep=' ', header=False, index=False)
pa[pa['mpa'].notnull()][['FID', 'IID']].to_csv('data/pheno/pa.m.keep', sep=' ', header=False, index=False)
pa[pa['fpa'].notnull()][['FID', 'IID']].to_csv('data/pheno/pa.f.keep', sep=' ', header=False, index=False)

In [10]:
%%bash

wc -l data/pheno/*pheno
wc -l data/pheno/*keep

  50332 data/pheno/pa.f.pheno
  39353 data/pheno/pa.m.pheno
  89684 data/pheno/pa.pheno
 179369 total
  50331 data/pheno/pa.f.keep
  89683 data/pheno/pa.keep
  39352 data/pheno/pa.m.keep
 179366 total
